## Topic Modeling with Gensim

pip install gensim
pip install pyldavis

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',
#메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

In [2]:
# 필요한 library들을 import
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [3]:
def tokenize(text):
    min_length = 3 #최소 단어 크기
    # nltk의 tokenizer를 이용해서 word 추출한 후에 소문자로 변환
    words = map(lambda word: word.lower(), word_tokenize(text))
    #words = word_tokenize(text.lower()) #이렇게 해도 되는지 확인
    # stopwords 제외
    words = [word for word in words if word not in cachedStopWords]
    # portr stemmer 적용
    tokens = (list(map(lambda token: PorterStemmer().stem(token),words)))
    #tokens = [PorterStemmer().stem(token) for token in words]  #이렇게 해도 되는지 확인
    #알파벳으로 이루어진 단어들만 추출
    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter (lambda token: p.match(token) and len(token) >= min_length,tokens))
    #filtered_tokens = [token for token in tokens if p.match(token) and len(token) >= min_length]   #이렇게 해도 되는지 확인
    return filtered_tokens

In [4]:
docs = [tokenize(doc) for doc in newsgroups_train.data]

In [5]:
print(newsgroups_train.data[0])
print(docs[0])

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
['notic', 'save', 'model', 'map', 'plane', 'posit', 'care', 'file', 'reload', 'restart', 'given', 'default', 'posit', 'orient', 'save', 'file', 'positions/orient', 'preserv', 'anyon', 'know', 'inform', 'store', 'file', 'noth', 'explicitli', 'said', 'manual', 'save', 'textur', 'rule', 'file', 'like', 'abl', 'read', 'textur', 'rule', 'inform', 'anyon', 'format', 'file', 'file', 'format', 'a

Gensim으로 topic modeling을 하기 위해서는 Dictionary, Corpus, Model의 3단계를 거쳐야 함

In [6]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.5)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 19839
Number of unique words after removing rare and common words: 5630


In [7]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5630
Number of documents: 2034


In [8]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 25
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 40
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 49.2 s


In [9]:
#! pip install pyldavis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### LDAvis
1) HTML의 왼쪽에 출력되는 topic의 2차원 embedding vector

비슷한 위치에 존재하는 토픽들은 서로 비슷한 문맥을 지니고 있다.

2) 오른쪽에 출력되는 각 토픽의 키워드

토픽은 단어 개수의 차원을 지니고 있다. 2차원으로 압축하기 위해서는 차원 축소 방법이 이용되어야한다. LDAvis는 Principal COmponent Analysis를 이용하여 n_terms 차원의 벡터들을 2차원으로 압축한다. PCA의 특성상 모든 토픽에 자주 등장하는 단어는 큰 정보력을 지니지 못한다. 토픽 간에 다른 패턴으로 등장하는 단어들을 중심으로 2차원의 지도의 좌표를 학습한다. 

In [11]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.240822 -0.098315       1        1  15.277046
8      0.015831  0.079881       2        1  10.709101
13     0.191753 -0.140678       3        1   8.523506
11     0.001669  0.247219       4        1   8.093948
2      0.149620 -0.149957       5        1   7.359865
10     0.144128 -0.045529       6        1   6.711622
20     0.164167  0.165431       7        1   5.709577
9      0.039985  0.236104       8        1   4.158284
4      0.147312  0.195078       9        1   3.954826
18    -0.150760  0.138067      10        1   3.326291
12    -0.075684  0.075257      11        1   3.176703
3     -0.015911  0.073982      12        1   2.923087
16    -0.062545  0.002726      13        1   2.845161
1      0.035670 -0.050129      14        1   2.736829
6      0.141142 -0.129801      15        1   2.570334
17     0.047416 -0.091360      16        1   2.471219
22     0.086696 -0.012158      17        1   2.410235
21    -0.067398 -0.061151      18        1   1.690786
14    -0.123006 -0.069808      19        1   1.016021
15    -0.080225 -0.056404      20        1   0.981462
7     -0.096171 -0.078458      21        1   0.951028
24    -0.164220 -0.082744      22        1   0.654919
19    -0.177711 -0.069545      23        1   0.628937
23    -0.191379 -0.037738      24        1   0.582317
0     -0.201203 -0.039970      25        1   0.536903, topic_info=         Term         Freq        Total Category  logprob  loglift
60        n't  1866.000000  1866.000000  Default  30.0000  30.0000
214     space   892.000000   892.000000  Default  29.0000  29.0000
617      year   474.000000   474.000000  Default  28.0000  28.0000
309      good   656.000000   656.000000  Default  27.0000  27.0000
123     think   823.000000   823.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
305     first    12.362479   425.786469  Topic25  -4.3474   1.6878
629    second     9.640675   108.517410  Topic25  -4.5960   2.8062
624     minut     8.180873    44.128933  Topic25  -4.7602   3.5418
886   raytrac     8.014616    41.515900  Topic25  -4.7808   3.5823
1690    built     8.079522    60.452644  Topic25  -4.7727   3.2146

[1485 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4245     15  0.939959      a.a
4377      8  0.065488    abort
4377     10  0.916828    abort
2476     20  0.942811  acceler
400       1  0.065614   accept
...     ...       ...      ...
617      18  0.094794     year
617      22  0.240145     year
789       2  0.952307     york
789       4  0.039679     york
4345     21  0.744504     zone

[3146 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 9, 14, 12, 3, 11, 21, 10, 5, 19, 13, 4, 17, 2, 7, 18, 23, 22, 15, 16, 8, 25, 20, 24, 1])